# 🎛️ Générateur de globalIndex.json pour Grime Index
Ce notebook lit tous les fichiers `*_data.json` dans le dossier `/content/data/` et génère un fichier `globalIndex.json` pour indexer artistes, instrumentales et archives.
Assure-toi que le dossier `data/` est bien présent dans l’environnement Colab, avec tes fichiers `.json` dedans.

In [ ]:
import json
import os
from pathlib import Path
from collections import defaultdict

# 📁 Chemin vers ton dossier data/
DATA_FOLDER = "/content/data"

# 📦 Structure du global index
index_data = {
    "speakers": defaultdict(list),
    "instrumentals": defaultdict(list),
    "archives": []
}

# 🔁 Parcourir tous les fichiers *_data.json
for filename in os.listdir(DATA_FOLDER):
    if filename.endswith("_data.json"):
        path = os.path.join(DATA_FOLDER, filename)
        with open(path, "r") as f:
            data = json.load(f)

        archive_id = Path(filename).stem.replace("_data", "")
        archive_title = data.get("title", archive_id)
        max_time = max((seg["end"] for seg in data["segments"]), default=0)

        # ➕ Ajouter archive
        index_data["archives"].append({
            "id": archive_id,
            "title": archive_title,
            "duration": round(max_time, 2)
        })

        # 🗣️ Indexer les speakers
        for seg in data.get("segments", []):
            speaker = seg.get("speaker", "unknown")
            if speaker:
                index_data["speakers"][speaker].append({
                    "archive": archive_id,
                    "start": round(seg["start"], 2),
                    "end": round(seg["end"], 2)
                })

        # 🎧 Indexer les instrumentals
        for instr in data.get("instrumentals", []):
            title = instr.get("title", "Untitled")
            if title:
                index_data["instrumentals"][title].append({
                    "archive": archive_id,
                    "start": round(instr["start"], 2),
                    "end": round(instr["end"], 2)
                })

# 💾 Sauvegarde dans globalIndex.json
output_path = os.path.join(DATA_FOLDER, "globalIndex.json")
with open(output_path, "w") as f:
    json.dump(index_data, f, indent=2)

print(f"✅ Fichier généré : {output_path}")
